# Tyre Quality Classification - Optimizers test

## Libraries & drive settings

In [ ]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,ConfusionMatrixDisplay,classification_report
import cv2
import matplotlib.image as mpimg
import sys
import datetime
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras import layers
import pathlib
import os
from google.colab import drive
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.optimizers.legacy import SGD
from tensorflow.keras.optimizers.legacy import RMSprop

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Data

## Loading (train and validation set)

In [ ]:
image_directory = "/content/drive/MyDrive/Tyre Quality Classification/data"
batch_size = 32
img_width = 224
img_height = 224

# Training data

train_data = tf.keras.utils.image_dataset_from_directory(image_directory,
                                                         validation_split=0.2,
                                                         seed=123,
                                                         subset="training",
                                                         image_size=(img_height, img_width),
                                                         batch_size=batch_size)

Found 1856 files belonging to 2 classes.
Using 1485 files for training.


In [ ]:
# Validation data

val_data = tf.keras.utils.image_dataset_from_directory(image_directory,
                                                       validation_split=0.2,
                                                       seed=123,
                                                       subset="validation",
                                                       image_size=(img_height, img_width),
                                                       batch_size=batch_size)

Found 1856 files belonging to 2 classes.
Using 371 files for validation.


In [ ]:
class_names = train_data.class_names
print(class_names)

['defective', 'good']


# Optimizers

## Adam

In [ ]:
num_classes = 2

model = tf.keras.Sequential([
    layers.experimental.preprocessing.Rescaling(1./255),
    layers.Conv2D(256,4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128,4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64,4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32,4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(16,4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

optimizer = Adam()

def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer._decayed_lr(tf.float32)
    return lr

lr_metric = get_lr_metric(optimizer)

model.compile(optimizer='adam', loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy', lr_metric])

# Callbacks (TensorBoard and early stopping to prevent overfitting)
logdir="/content/drive/MyDrive/Tyre Quality Classification/logs"
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir,histogram_freq=1, write_images=logdir, embeddings_data=train_data)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(train_data,
          validation_data=val_data,
          epochs=20,
          callbacks=[tensorboard_callback, early_stopping]
)

Epoch 1/20
47/47 [==============================] - 65s 1s/step - loss: 0.6915 - accuracy: 0.5300 - lr: 1.0000e-03 - val_loss: 0.6874 - val_accuracy: 0.5660 - val_lr: 0.0010
Epoch 2/20
47/47 [==============================] - 62s 1s/step - loss: 0.6879 - accuracy: 0.5508 - lr: 1.0000e-03 - val_loss: 0.6820 - val_accuracy: 0.5687 - val_lr: 0.0010
Epoch 3/20
47/47 [==============================] - 63s 1s/step - loss: 0.6837 - accuracy: 0.5461 - lr: 1.0000e-03 - val_loss: 0.6508 - val_accuracy: 0.6658 - val_lr: 0.0010
Epoch 4/20
47/47 [==============================] - 63s 1s/step - loss: 0.6652 - accuracy: 0.6141 - lr: 1.0000e-03 - val_loss: 0.7657 - val_accuracy: 0.4313 - val_lr: 0.0010
Epoch 5/20
47/47 [==============================] - 62s 1s/step - loss: 0.6406 - accuracy: 0.6316 - lr: 1.0000e-03 - val_loss: 0.5683 - val_accuracy: 0.7035 - val_lr: 0.0010
Epoch 6/20
47/47 [==============================] - 63s 1s/step - loss: 0.6092 - accuracy: 0.6505 - lr: 1.0000e-03 - val_loss: 0.6

In [ ]:
loss, accuracy, lr = model.evaluate(val_data)
print(f"Accuracy:{accuracy}")
print(f"Loss:{loss}")

12/12 [==============================] - 13s 262ms/step - loss: 0.5763 - accuracy: 0.7116 - lr: 0.0010
Accuracy:0.7115902900695801
Loss:0.5763291716575623


In [ ]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_11 (Rescaling)    (None, 224, 224, 3)       0         
                                                                 
 conv2d_55 (Conv2D)          (None, 221, 221, 256)     12544     
                                                                 
 max_pooling2d_55 (MaxPooli  (None, 110, 110, 256)     0         
 ng2D)                                                           
                                                                 
 conv2d_56 (Conv2D)          (None, 107, 107, 128)     524416    
                                                                 
 max_pooling2d_56 (MaxPooli  (None, 53, 53, 128)       0         
 ng2D)                                                           
                                                                 
 conv2d_57 (Conv2D)          (None, 50, 50, 64)      

## SGD

In [ ]:
num_classes = 2

model = tf.keras.Sequential([
    layers.experimental.preprocessing.Rescaling(1./255),
    layers.Conv2D(256,4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128,4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64,4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32,4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(16,4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

optimizer = SGD()

def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer._decayed_lr(tf.float32)
    return lr

lr_metric = get_lr_metric(optimizer)

model.compile(optimizer=optimizer, loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy', lr_metric])

# Callbacks (TensorBoard and early stopping to prevent overfitting)
logdir="/content/drive/MyDrive/Tyre Quality Classification/logs"
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir,histogram_freq=1, write_images=logdir, embeddings_data=train_data)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(train_data,
          validation_data=val_data,
          epochs=20,
          callbacks=[tensorboard_callback, early_stopping]
)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


47/47 [==============================] - 65s 1s/step - loss: 0.6905 - accuracy: 0.5508 - lr: 0.0100 - val_loss: 0.6859 - val_accuracy: 0.5660 - val_lr: 0.0100
Epoch 2/20
47/47 [==============================] - 71s 1s/step - loss: 0.6854 - accuracy: 0.5522 - lr: 0.0100 - val_loss: 0.6805 - val_accuracy: 0.5660 - val_lr: 0.0100
Epoch 3/20
47/47 [==============================] - 71s 1s/step - loss: 0.6827 - accuracy: 0.5582 - lr: 0.0100 - val_loss: 0.6781 - val_accuracy: 0.5660 - val_lr: 0.0100
Epoch 4/20
47/47 [==============================] - 63s 1s/step - loss: 0.6790 - accuracy: 0.5657 - lr: 0.0100 - val_loss: 0.6833 - val_accuracy: 0.6658 - val_lr: 0.0100
Epoch 5/20
47/47 [==============================] - 63s 1s/step - loss: 0.6815 - accuracy: 0.5670 - lr: 0.0100 - val_loss: 0.6716 - val_accuracy: 0.5849 - val_lr: 0.0100
Epoch 6/20
47/47 [==============================] - 62s 1s/step - loss: 0.6768 - accuracy: 0.5663 - lr: 0.0100 - val_loss: 0.6878 - val_accuracy: 0.4717 - val_lr

In [ ]:
loss, accuracy, lr = model.evaluate(val_data)
print(f"Accuracy:{accuracy}")
print(f"Loss:{loss}")

12/12 [==============================] - 11s 258ms/step - loss: 0.5800 - accuracy: 0.7035 - lr: 0.0100
Accuracy:0.7035040259361267
Loss:0.5799956321716309


In [ ]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_5 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 conv2d_25 (Conv2D)          (None, 221, 221, 256)     12544     
                                                                 
 max_pooling2d_25 (MaxPooli  (None, 110, 110, 256)     0         
 ng2D)                                                           
                                                                 
 conv2d_26 (Conv2D)          (None, 107, 107, 128)     524416    
                                                                 
 max_pooling2d_26 (MaxPooli  (None, 53, 53, 128)       0         
 ng2D)                                                           
                                                                 
 conv2d_27 (Conv2D)          (None, 50, 50, 64)       

## RMSprop

In [ ]:
num_classes = 2

model = tf.keras.Sequential([
    layers.experimental.preprocessing.Rescaling(1./255),
    layers.Conv2D(256,4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128,4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64,4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32,4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(16,4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

optimizer = RMSprop()

def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer._decayed_lr(tf.float32)
    return lr

lr_metric = get_lr_metric(optimizer)

model.compile(optimizer=optimizer, loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy', lr_metric])

# Callbacks (TensorBoard and early stopping to prevent overfitting)
logdir="/content/drive/MyDrive/Tyre Quality Classification/logs"
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir,histogram_freq=1, write_images=logdir, embeddings_data=train_data)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(train_data,
          validation_data=val_data,
          epochs=20,
          callbacks=[tensorboard_callback, early_stopping]
)

Epoch 1/20
47/47 [==============================] - 64s 1s/step - loss: 0.7071 - accuracy: 0.5320 - lr: 1.0000e-03 - val_loss: 0.6881 - val_accuracy: 0.5660 - val_lr: 0.0010
Epoch 2/20
47/47 [==============================] - 63s 1s/step - loss: 0.6928 - accuracy: 0.5529 - lr: 1.0000e-03 - val_loss: 0.6913 - val_accuracy: 0.5660 - val_lr: 0.0010
Epoch 3/20
47/47 [==============================] - 62s 1s/step - loss: 0.6895 - accuracy: 0.5468 - lr: 1.0000e-03 - val_loss: 0.6877 - val_accuracy: 0.5660 - val_lr: 0.0010
Epoch 4/20
47/47 [==============================] - 62s 1s/step - loss: 0.6937 - accuracy: 0.5461 - lr: 1.0000e-03 - val_loss: 0.6872 - val_accuracy: 0.5660 - val_lr: 0.0010
Epoch 5/20
47/47 [==============================] - 63s 1s/step - loss: 0.7504 - accuracy: 0.5811 - lr: 1.0000e-03 - val_loss: 0.6702 - val_accuracy: 0.6604 - val_lr: 0.0010
Epoch 6/20
47/47 [==============================] - 63s 1s/step - loss: 0.6635 - accuracy: 0.6114 - lr: 1.0000e-03 - val_loss: 0.6

In [ ]:
loss, accuracy, lr = model.evaluate(val_data)
print(f"Accuracy:{accuracy}")
print(f"Loss:{loss}")

12/12 [==============================] - 13s 257ms/step - loss: 0.6012 - accuracy: 0.7008 - lr: 0.0010
Accuracy:0.7008086442947388
Loss:0.6012292504310608


In [ ]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_10 (Rescaling)    (None, 224, 224, 3)       0         
                                                                 
 conv2d_50 (Conv2D)          (None, 221, 221, 256)     12544     
                                                                 
 max_pooling2d_50 (MaxPooli  (None, 110, 110, 256)     0         
 ng2D)                                                           
                                                                 
 conv2d_51 (Conv2D)          (None, 107, 107, 128)     524416    
                                                                 
 max_pooling2d_51 (MaxPooli  (None, 53, 53, 128)       0         
 ng2D)                                                           
                                                                 
 conv2d_52 (Conv2D)          (None, 50, 50, 64)      

RMS better than SGD because it reach the same accuracy with less epoch (early stop), so more efficient.
Adam 72% in only 8 epoch.